<a href="https://colab.research.google.com/github/esnue/ThesisAllocationSystem/blob/main/contextualized_topic_modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tutorial: Contextualized Topic Models with Wikipedia Documents

(last updated 11-01-2021)

In this tutorial, we are going to use contextualized topic modeling to get topics out of a collections of articles you will upload here.

## Topic Models 

Topic models allow you to discover latent topics in your documents in a completely unsupervised way. Just use your documents and get topics out.

## Contextualized Topic Models


What are Contextualized Topic Models? **CTMs** are a family of topic models that combine the expressive power of BERT embeddings with the unsupervised capabilities of topic models to get topics out of documents.

## Python Package

You can find our package [here](https://github.com/MilaNLProc/contextualized-topic-models).

![https://travis-ci.com/MilaNLProc/contextualized-topic-models](https://travis-ci.com/MilaNLProc/contextualized-topic-models.svg) ![https://pypi.python.org/pypi/contextualized_topic_models](https://img.shields.io/pypi/v/contextualized_topic_models.svg) ![https://pepy.tech/badge/contextualized-topic-models](https://pepy.tech/badge/contextualized-topic-models)




# Enabling the GPU

First, you'll need to enable GPUs for the notebook:

- Navigate to Edit→Notebook Settings
- select GPU from the Hardware Accelerator drop-down

[Reference](https://colab.research.google.com/notebooks/gpu.ipynb)

# Installing Contextualized Topic Models

Now, we install the contextualized topic model library

In [4]:
%%capture
!pip install contextualized-topic-models==1.8.1
!pip install torch==1.6.0+cu101 torchvision==0.7.0+cu101 -f https://download.pytorch.org/whl/torch_stable.html

## Data

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
import pandas as pd

file = pd.read_csv("/content/drive/MyDrive/ThesisAllocationSystem/data_final/train-papers-final.csv")
#transform content column row values into a list
train_file = file['Content'][1:].values.tolist()

print(train_file[5])
print(type(text_file))

file2 = pd.read_csv("/content/drive/MyDrive/ThesisAllocationSystem/data_final/test-theses-final.csv")
test_file = file2['Content'][1:].values.tolist()

print(test_file[5])
print(type(test_file))

b"Public Administration Reform in Europe  \n\nViews and Experiences from Senior Executives in 10 Countries \n\n \nWP3 Cross-national Report as part of the COCOPS Research Project \n\n \n\n  \n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\nGerhard Hammerschmid \nSteven Van de Walle \n\nRhys Andrews \nPhilippe Bezes \nAnja G\xf6rnitz \nAnca Oprisor \n\nVid timac \n\n \n\nJuly 2013 \n\n \n\n \n\n \n\n \n\n \n\nCoordination for Cohesion in the Public Sector of the Future (COCOPS): www.cocops.eu \n\n  \n\n \n\nhttp://www.cocops.eu/\n\n\nCOCOPS Work Package 3  Cross-national Report, July 2013 Page 2 \n\nAbout COCOPS \n\nThe COCOPS project (Coordinating for Cohesion in the Public Sector of the Future) seeks to \n\ncomparatively and quantitatively assess the impact of New Public Management-style reforms in \n\nEuropean countries, drawing on a team of European public administration scholars from 11 \n\nuniversities in 10 countries. It analyses the impact of reforms i

## Importing what we need

In [8]:
from contextualized_topic_models.models.ctm import CombinedTM
from contextualized_topic_models.utils.data_preparation import bert_embeddings_from_list, TopicModelDataPreparation
from contextualized_topic_models.datasets.dataset import CTMDataset
from contextualized_topic_models.evaluation.measures import CoherenceNPMI, InvertedRBO
from gensim.corpora.dictionary import Dictionary
from gensim.test.utils import common_texts
from gensim.models import ldamodel 
from contextualized_topic_models.utils.preprocessing import WhiteSpacePreprocessing
import nltk
import os
import pickle

## Preprocessing

Let's pass our file with preprocess data to our text handler object. This object takes care of creating the bag of words for you.

Why do we use the **preprocessed text** here? we need text without punctuation to build the bag of word. Also, we might want only to have the most frequent words inside the BoW. Too many words might not help.

In [11]:
nltk.download('stopwords')

documents = train_file
#documents = [line.strip() for line in open(text_file, encoding="utf-8").readlines()]
sp = WhiteSpacePreprocessing(documents, "english")
train_preprocessed_documents_for_bow, train_unpreprocessed_corpus_for_contextual, vocab = sp.preprocess()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [12]:
documents = test_file
#documents = [line.strip() for line in open(text_file, encoding="utf-8").readlines()]
sp = WhiteSpacePreprocessing(documents, "english")
test_preprocessed_documents_for_bow, test_unpreprocessed_corpus_for_contextual, vocab = sp.preprocess()

You might want to pickle the **training_dataset** object to avoid recomputing the BoW multiple times

In [13]:
qt = TopicModelDataPreparation("bert-base-nli-mean-tokens")

training_dataset = qt.create_training_set(train_unpreprocessed_corpus_for_contextual, train_preprocessed_documents_for_bow)


100%|██████████| 405M/405M [00:10<00:00, 38.3MB/s]


Batches:   0%|          | 0/5 [00:00<?, ?it/s]

## Training our Contextualized Topic Model

Finally, we can fit our new topic model. We will ask the model to find 50 topics in our collection (n_component parameter of the CTM object).

In [14]:
ctm = CombinedTM(input_size=len(qt.vocab), bert_input_size=768, n_components=25, num_epochs=15)

ctm.fit(training_dataset) 

Epoch: [15/15]	 Seen Samples: [12150/12150]	Train Loss: 13186.887808641975	Time: 0:00:00.483489: : 15it [00:07,  2.00it/s]


In [19]:
path = F"/content/drive/MyDrive/ThesisAllocationSystem/models/ctm.bin" 
ctm.save(path)

/usr/local/lib/python3.7/dist-packages/contextualized_topic_models/models/ctm.py:388: Warning: This is an experimental feature that we has not been fully tested. Refer to the following issue:https://github.com/MilaNLProc/contextualized-topic-models/issues/38
  Warning)


# Topics

After training, now it is the time to look at our topics: we can use the 

```
get_topic_lists
```

function to get the topics. It also accepts a parameter that allows you to select how many words you want to see for each topic.

If you look at the topics, you will see that they all make sense and are representative of a collection of documents that comes from Wikipedia (general knowledge).

In [20]:
ctm.get_topic_lists(5)

[['starting', 'equally', 'reach', 'perhaps', 'successful'],
 ['social', 'organizational', 'organizations', 'conflict', 'field'],
 ['economic', 'education', 'based', 'system', 'training'],
 ['firms', 'data', 'per', 'time', 'large'],
 ['making', 'actors', 'different', 'governance', 'nand'],
 ['germany', 'german', 'fertility', 'first', 'women'],
 ['gain', 'essential', 'nsome', 'class', 'define'],
 ['law', 'european', 'eds', 'rights', 'convention'],
 ['high', 'market', 'paper', 'different', 'nto'],
 ['economic', 'vote', 'price', 'energy', 'wind'],
 ['public', 'policy', 'administrative', 'political', 'government'],
 ['rights', 'human', 'law', 'eu', 'nthe'],
 ['political', 'government', 'governments', 'level', 'financial'],
 ['circumstances', 'allowing', 'forward', 'changing', 'refer'],
 ['data', 'results', 'time', 'using', 'used'],
 ['social', 'development', 'research', 'management', 'activities'],
 ['market', 'power', 'price', 'costs', 'energy'],
 ['media', 'political', 'government', 'new'

### Let's find our documents' topics

Ok now we can take a document and see which topic has been asigned to it. Results will obviously change with respect to the documents you are using. For example, let's predict the topic of the first preprocessed document that is taling about a peninsula.

In [21]:
topics_predictions = ctm.get_doc_topic_distribution(training_dataset, n_samples=20) # get all the topic predictions

Sampling: [20/20]: : 20it [00:08,  2.24it/s]


In [23]:
train_preprocessed_documents_for_bow[0] # see the text of our preprocessed document

'nsee discussions author publication https www researchgate net publication incentive preferences issues strategies local government first evidence narticle authors including nsome authors publication also working related projects open innovative governments view project public sector future view project publications nsee profile school governance publications nsee profile nall content following page september nthe user downloaded nhttps www researchgate net publication motivation incentive preferences issues strategies local government first evidence austria enrichid rgreq xxx enrichsource el esc publicationcoverpdf nhttps www researchgate net publication motivation incentive preferences issues strategies local government first evidence austria enrichid rgreq xxx enrichsource el esc publicationcoverpdf nhttps www researchgate net project open innovative governments enrichid rgreq xxx enrichsource el esc publicationcoverpdf nhttps www researchgate net project public sector future enric

In [24]:
import numpy as np
topic_number = np.argmax(topics_predictions[0]) # get the topic id of the first document
topic_number

4

In [25]:
ctm.get_topic_lists(5)[topic_number] #and the topic should and could be about public management related things

['making', 'actors', 'different', 'governance', 'nand']

## Test

We are using the test set to predict for unseen documents. 

In [28]:
testing_dataset = qt.create_test_set(test_unpreprocessed_corpus_for_contextual, test_preprocessed_documents_for_bow) # create dataset for the testset
predictions = ctm.get_doc_topic_distribution(testing_dataset, n_samples=10)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Sampling: [10/10]: : 10it [00:01,  5.34it/s]


In [33]:
print(test_unpreprocessed_corpus_for_contextual[2])

topic_index = np.argmax(predictions[2])
ctm.get_topic_lists(5)[topic_index]

b"I will attend: \n\nName\n\nPosition\n\nInstitution\n\nEmail \n\nAccompanied by\n\nRegistration\n\nStudent Paper Series\nThe EUs Extraterritorial Asylum Policies \n\n& Human Rights Obligations:\nProspects, Limits and the Role of the European Courts \n\nBy Anna Hoffmann (MPP 2015)\n\nAcademic Advisor: Prof. Mark Dawson, PhD\nProfessor of European Law and Governance\n\n \nHSSPS 01 | 2015\n\n\n\nAcknowledgements \n \n\nOne incurs many debts even when writing a rather short paper. I would like to thank \n\nmy supervisor Mark Dawson for the helpful comments and advice he provided in \n\nframing the research topic. This paper would not have emerged in this form without \n\nthe stimulating debates and insights provided in the course he co-taught with Dilek \n\nKurban. I am also grateful to Franziska Vilmar from Amnesty International for a \n\nproductive conversation about some of the topics covered in this thesis. My wonderful \n\nfriends were very supportive and Florien, Nick, Julia, Max, N

['rights', 'human', 'law', 'eu', 'nthe']